## Classifying unstructured geographic text based on distance

This experiment employs LLMs to extract and summarize geographic information from unstructured, empirical literature on migration studies, especially climate migration (i.e., climate change or environmental events related human migration). The core task here is to simply, automatically classify the type of human migration discussed in each paper into 2 qualitative categories: long-distance human migration or short-distance human migration. 

It's important to note that this task differs from more straightforward NER, but relatively comprehensive. The conceptualization of migration type is rarely explicitly addressed by the context, but should be inferred by LLMs based on more implicit natural language descriptions. According to the literature, apart from NER-like extraction, such information should sometimes be inferred from distance between place names, from difficulties induced by border crossing, from other synonyms or semantically similar words like 'internal human migration' and 'international human migration' and so on and so forth. It's natually that such conceptualization can't be identical when different background applies, however, studying that is not useless, since it not only helps data analysis, ontology engineering and information retrieval, but also influence decision making and policy making regarding human migration. 

### Libraries

In [118]:
from openai import OpenAI, RateLimitError, APIError, APITimeoutError, AuthenticationError, BadRequestError, NotFoundError
from google.api_core.exceptions import ResourceExhausted, RetryError, DeadlineExceeded
import google.generativeai as genai

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# from multiprocessing import Pool
import time
import os
import ast
from tqdm import tqdm
import sklearn

### Layout

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_items', None)

### Data

In this experiment we use pre-processed literature, namely literature processed as Python Dictionary, to serve as our input, since directly inputting PDF is not supported by some of the LLMs. To ensure equality (since we do not aim to compare performance among different LLMs here), we simply make this decision. For transferring PDF to Dictionary please refer to another file called pdf2text.ipynb. We admit that this may cause lower accuracy as figures and tables are excluded, and context can be incoherent.

In [4]:
literature = np.load('literature.npy', allow_pickle=True).item()
shot = np.load('example.npy', allow_pickle=True).item()

### Pre-defined variables

Anything you want to set as prior, such as max token, temperature, top-p, logprob, etc.

In [7]:
MAX_TOKEN = 16

### General commands for getting response

We employ 3 different models: gpt-4.1-mini, gemini-2.5-flash, and deepseek-3.2-chat via API provided by them. For each model, we consider 3 scenarios: 0-shot, 1-shot, and 3-shot, which enables us to see if giving examples can make any difference. Giving examples to LLMs can make them perform better is also a hypothesis of our research.

In [44]:
def api_def(provider):
    # API keys are hidden in another file
    with open('api.txt', 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file.readlines()]

        # Different LLMs need different kinds of command
        if provider == 'chatgpt':
            client = OpenAI(api_key=str(lines[0]), base_url="https://api.openai.com/v1")
            return client
        
        elif provider == 'deepseek':
            client = OpenAI(api_key=str(lines[1]), base_url="https://api.deepseek.com")
            return client
        
        elif provider == 'gemini':
            genai.configure(api_key=str(lines[4]))

In [29]:
def get_completion(prompt, provider, model, temperature):
    messages = [{"role": "system", "content": "You are an expert in Human Geography, synthesizing literature on \
    climate-induced migration. Your task is to simply classify the provided text, solely based on the evidence in it."},
               {"role": "user", "content": prompt}]

    # Different LLMs need different kinds of command
    if provider == 'gpt':
        client = api_def('chatgpt')
        response = client.responses.create(
            model=model,
            input=messages,
            temperature=temperature,
            max_output_tokens = MAX_TOKEN
        )
        return response.output_text
        
    elif provider == 'ds':
        client = api_def('deepseek')
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens = MAX_TOKEN
        )
        return response.choices[0].message.content

    elif provider == 'gemini':
        api_def('gemini')
        model_gemini = genai.GenerativeModel(model)
        response = model_gemini.generate_content(
            prompt,
            generation_config={"temperature": temperature, "max_output_tokens": MAX_TOKEN * 5}
        )
        return response.text        

In [9]:
CONTEXT = f"""
We are classifying academic papers based on the type of human migration they address. \n \n
While reviewing a paper, assign it one of the following codes based solely on the evidence presented in this specific paper: \n
'1': The paper explicitly focuses on or observes evidence for long-distance human migration. \n
'2': The paper explicitly focuses on or observes evidence for short-distance human migration. \n
'3': The paper explicitly focuses on or observes evidence for both long-distance and short-distance human migration. \n
'0': The paper does not provide any explicit information on distance of human migration. \n \n

Instructions for doing classification: \n
1.  While classifying a paper, focus only on its own content. Do not infer any missing information from e.g., past \
tasks. Only concentrate on sections that explicitly describe this exact work, such as 'abstract', 'objectives', \
'data', 'methods', 'results', and 'conclusions', etc., rather than those that primarily summarize others' \
work, like 'introduction', 'literature review' or 'related works' etc. \n
2.  Classification of migration type should be based on interpreting context. This includes not only extracting \
explicit mentions of 'long-distance human migration' or 'short-distance human migration', but also reasoning \
according to the context where for example, migration distance, origins and destinations, borders crossing etc. \
are addressed. \n \n

Return only the single character without any other thing such as explanations, opinions or meta-comments. 
"""

In [10]:
def get_response(paper: dict, example, provider, model, temperature):

    fail_count = 0
    while True:
        if example == 3:
            prompt = f"""
            <Group leader>: {CONTEXT} \n
            Now please assign a character for the following paper: \n
            \"\"\"{shot[0]}\"\"\"
            \n
            <Expert>: My answer is: 1 \n
            <Group leader>: Please assign a character for another paper: \n
            \"\"\"{shot[1]}\"\"\"
            \n
            <Expert>: My answer is: 3 \n
            <Group leader>: Please assign a character for another paper: \n
            \"\"\"{shot[2]}\"\"\"
            \n
            <Expert>: My answer is: 2 \n
            
            <Group leader>: Please assign a character for this paper like what the expert did: \n"
            f"{paper}\n"
            "Keep in mind that output is only one character."
            """

        elif example == 1:
            prompt = f"""
            <Group leader>: {CONTEXT} \n
            Now please assign a character for the following paper: \n
            \"\"\"{shot[0]}\"\"\"
            \n
            <Expert>: My answer is: 1 \n
            
            <Group leader>: Please assign a character for this paper like what the expert did: \n"
            f"{paper}\n"
            "Keep in mind that output is only one character."
            """

        elif example == 0:
            prompt = f"""
            <Group leader>: {CONTEXT} \n
            Please assign a character for the following paper: \n
            f"{paper}\n"
            "Keep in mind that output is only one character."
            """

        response = get_completion(prompt, provider, model, temperature)
        return response

### Organization

For each model and each scenario, we consider 5 different temperatures and run each experiment for 10 times to test the influence of temperature parameter. The whole process is applied to 30 different papers in the same way.

In [39]:
def response_agg(literature, example, provider, model, idx):
    results = []
    fail_count = 0
    temperatures = np.round(np.arange(0.0, 2.0, 0.4), 1)
        
    for t in tqdm(temperatures, desc=f"File {idx+1}"):        
        for run_id in range(10):
            success = False
            while not success:
                try:
                    result = {"answer": get_response(literature, example, provider, model, t)}
                    result["provider"] = provider
                    result["model"] = model
                    result["few shot"] = example
                    result["temperature"] = t
                    result["run"] = run_id
                    results.append(result)
                    # When exceed Tokens per minute
                    time.sleep(1)
                    success = True
                except (RateLimitError, APIError, APITimeoutError, ResourceExhausted) as e: # AuthenticationError, BadRequestError, NotFoundError
                    time.sleep(1)
                    fail_count = fail_count + 1
                    if fail_count >= 10:
                        raise ValueError(f"Can't get response, retried too many times") from e

    df = pd.DataFrame(results)
    return df

In [13]:
def batch_response_to_df(all_literature, example, provider, model):

    save_path = f"results_lit/{provider}_{example}.csv"
    os.makedirs("results_lit", exist_ok=True)

    if os.path.exists(save_path):
        temporal_df = pd.read_csv(save_path)
        processed_ids = set(temporal_df["file_id"].astype(int).tolist())
        print(f"Start from paper {len(processed_ids)+1}")
    else:
        temporal_df = pd.DataFrame()
        processed_ids = set()
        
    all_results = [temporal_df] if not temporal_df.empty else []
    
    for idx, literature in enumerate(tqdm(all_literature.values(), desc="Progress")):
        file_id = idx + 1
        if file_id in processed_ids:
            continue
        
        df = response_agg(literature, example, provider, model, idx)
        df["file_id"] = file_id
        all_results.append(df)

        temporal_df = pd.concat(all_results, ignore_index=True)
        temporal_df.astype(str).to_csv(save_path, index=False)

    final_df = pd.concat(all_results, ignore_index=True)
    return final_df

In all: 30 documents, 3 models, 3 scenarios regarding amount of examples, 5 temperatures, 10 runs. In sum we need to call api for 13500 times, and will get a final result containing 13500 rows.

### Results

In [14]:
gpt41_0 = batch_response_to_df(literature, 0, 'gpt', 'gpt-4.1-mini')

Progress: 100%|██████████| 30/30 [25:01<00:00, 50.05s/it]


In [15]:
gpt41_1 = batch_response_to_df(literature, 1, 'gpt', 'gpt-4.1-mini')

Progress: 100%|██████████| 30/30 [26:24<00:00, 52.81s/it]


In [37]:
gpt41_3 = batch_response_to_df(literature, 3, 'gpt', 'gpt-4.1-mini')

Start from paper 7


Progress: 100%|██████████| 30/30 [30:14<00:00, 60.49s/it]


In [17]:
ds32_0 = batch_response_to_df(literature, 0, 'ds', 'deepseek-chat')

Progress: 100%|██████████| 30/30 [42:12<00:00, 84.43s/it]


In [18]:
ds32_1 = batch_response_to_df(literature, 1, 'ds', 'deepseek-chat')

Progress: 100%|██████████| 30/30 [45:08<00:00, 90.29s/it]


In [19]:
ds32_3 = batch_response_to_df(literature, 3, 'ds', 'deepseek-chat')

Progress: 100%|██████████| 30/30 [53:48<00:00, 107.60s/it]


In [23]:
gemini25_0 = batch_response_to_df(literature, 0, 'gemini', 'gemini-2.5-flash')

Progress: 100%|██████████| 30/30 [30:04<00:00, 60.16s/it]


In [24]:
gemini25_1 = batch_response_to_df(literature, 1, 'gemini', 'gemini-2.5-flash')

Progress: 100%|██████████| 30/30 [31:49<00:00, 63.65s/it]


In [45]:
gemini25_3 = batch_response_to_df(literature, 3, 'gemini', 'gemini-2.5-flash')

Start from paper 23


Progress: 100%|██████████| 30/30 [15:34<00:00, 31.16s/it]


### Evaluation

We simply calculate the accuracy of classification results compared to human annotation first. We get an accuracy value for each setting regarding model, few-shot scenario and temperature.

In [119]:
manual_class = pd.read_excel('manual.xlsx')
manual_class["class"] = (manual_class["Long distance"].astype(int) + manual_class["Short distance"].astype(int)*2).astype(str)
manual_class = manual_class.drop(index=range(0, 3), columns=["ID", "AUTHOR", "TITLE", "Long distance", "Short distance"]).reset_index(drop=True)

In [130]:
df_dict = {}

for file_name in os.listdir('results_lit/'):
    if file_name.endswith('.csv'):
        file_path = os.path.join('results_lit/', file_name)

        df = pd.read_csv(file_path)
        df_name = file_name.replace('.csv', '')
        df_dict[df_name] = df

LLM_class = pd.concat(df_dict.values(), axis=0, ignore_index=True)
LLM_class['file_id'] = LLM_class['file_id'].astype(int) - 1
LLM_class['label'] = (LLM_class['file_id'].map(manual_class['class'])).astype(str).str.strip()
LLM_class['answer'] = LLM_class['answer'].astype(str).str.strip()

In [133]:
LLM_class['accuracy'] = (LLM_class['label'] == LLM_class['answer']).astype(int)
LLM_new = LLM_class.drop(columns=['provider', 'answer', 'label']).reset_index(drop=True)
LLM_summary = LLM_new.groupby(['model', 'few shot', 'temperature']).agg({'accuracy': 'mean'}).reset_index()
LLM_summary

,model,few shot,temperature,accuracy
0,deepseek-chat,0,0.0,0.273333
1,deepseek-chat,0,0.4,0.286667
2,deepseek-chat,0,0.8,0.286667
3,deepseek-chat,0,1.2,0.280000
4,deepseek-chat,0,1.6,0.276667
5,deepseek-chat,1,0.0,0.233333
6,deepseek-chat,1,0.4,0.243333
7,deepseek-chat,1,0.8,0.256667
8,deepseek-chat,1,1.2,0.223333
9,deepseek-chat,1,1.6,0.246667


In [136]:
# random
file_ids = np.repeat(np.arange(30), 10)
random_class = np.random.randint(0, 4, size=300)

random = pd.DataFrame({
    'file_id': file_ids,
    'answer': random_class
})

In [137]:
random['label'] = (random['file_id'].map(manual_class['class'])).astype(str).str.strip()
random['answer'] = random['answer'].astype(str).str.strip()
random['accuracy'] = (random['label'] == random['answer']).astype(int)
random = random.drop(columns=['answer']).reset_index(drop=True)
random_summary = random['accuracy'].mean()
random_summary

np.float64(0.26)

In [138]:
def plot(df, model, few_shot, pos):    
    subset = df[(df['model'] == model)]
    
    plt.figure(figsize=(8,6))
    plt.title(f"{model} - Accuracy", fontsize=24, pad=15)

    few_shot_values = subset['few shot'].unique()
    colors = {'gpt-4.1-mini': '#036aff', 'deepseek-chat': '#9cafff', 'gemini-2.5-flash': '#ff7a40'}
    color = colors.get(model, '#036aff')
    line_styles = ['-', '--', '-.']

    plt.axhline(y=random_summary, color='black', linestyle=':', linewidth=2, label='random')
    for i, few_shot in enumerate(few_shot_values):
        few_shot_data = subset[subset['few shot'] == few_shot]
        line_style = line_styles[i % len(line_styles)]
        plt.plot(few_shot_data['temperature'], few_shot_data['accuracy'], marker='o', markersize=6, linewidth=2, \
                color=color, linestyle=line_style, label=f'{few_shot}-shot')
    
    plt.xlabel('Temperature', fontsize=18)
    plt.ylabel('Accuracy', fontsize=18)
    plt.grid(True, alpha=0.3)
    plt.ylim(0.2, 0.6)
    plt.xticks(np.arange(0, 2, 0.4), fontsize=14)
    plt.yticks(np.arange(0.2, 0.7, 0.1), fontsize=14)
    plt.legend(title='Scenario', title_fontsize=18, fontsize=14, loc='best', framealpha=0.9)
    filepath = os.path.join(pos, f"{model}.png")
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close()

In [139]:
for model in LLM_summary['model'].unique():
    for few_shot in LLM_summary['few shot'].unique():
        plot(LLM_summary, model, few_shot, 'plots/lit')

LLM classification results show marked discrepancies from human annotations. In the case of DeepSeek specifically, accuracy remains indistinguishable from random chance across various scenarios and temperature settings. Let's look at confusion matrix a bit.

In [148]:
def conf_matrix(df, model, few_shot, temperature):
    subset = df[(df['model'] == model) & (df['few shot'] == few_shot) & (df['temperature'] == temperature)]
    matrix = pd.DataFrame(sklearn.metrics.confusion_matrix(subset['label'], subset['answer']))
    return matrix

In [149]:
for model in LLM_class['model'].unique():
    for few_shot in LLM_class['few shot'].unique():
        for temperature in LLM_class['temperature'].unique():
            conf_matrix(LLM_class, model, few_shot, temperature).to_csv(f"analysis_lit/{model}_{few_shot}_{temperature}.csv", index=False)

In [150]:
sklearn.metrics.confusion_matrix(LLM_class['label'], LLM_class['answer'])

array([[ 773,  363,  815,  749],
       [ 161,  699,   24,  466],
       [2117,  833, 2053, 1297],
       [ 980,  375,  409, 1386]])

In [154]:
subset = LLM_class[(LLM_class['model'] == 'gpt-4.1-mini')]
sklearn.metrics.confusion_matrix(subset['label'], subset['answer'])

array([[  65,  130,  361,  344],
       [   0,  251,   22,  177],
       [ 163,  233, 1193,  511],
       [ 108,  120,  265,  557]])